In [47]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from IPython.display import display, HTML
from typing import List
from mprompt.modules.emb_diff_module import EmbDiffModule
import numpy as np
import matplotlib
import imodelsx.util
from copy import deepcopy
import re
import notebook_helper
import mprompt.viz
import scipy.special
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import pandas as pd
# from mprompt.config import RESULTS_DIR
import torch.cuda
import json
from sklearn.linear_model import RidgeCV
import pandas as pd
import re
import pyLDAvis
import pyLDAvis.lda_model
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
RESULTS_DIR = '/home/chansingh/mprompt/results/bert'
df = pd.read_csv(join(RESULTS_DIR, "dict_learn_results - extracted factors_all.csv"))
short_strings = df['top_explanation_init_strs'].dropna().values.tolist()

In [49]:
# Preprocessing: remove numbers, special characters, and lowercase all words
processed_docs = [re.sub(r'[^\w\s]', '', doc.lower()) for doc in short_strings]

# Add custom stop words
# additional_stop_words = ['dont', 'doesnt', 'didnt', 'couldnt', 'wont', 'cant']
# stop_words = text.ENGLISH_STOP_WORDS.union(additional_stop_words)

# Vectorize the text data
vectorizer = CountVectorizer(
    stop_words='english',
    max_df=0.95, min_df=2)
dtm = vectorizer.fit_transform(processed_docs)

# LDA model
num_topics = 10
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(dtm)

# Display topics
words = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic #{topic_idx + 1}:")
    print(" ".join([words[i] for i in topic.argsort()[:-11:-1]]))

Topic #1:
words technology related media phrases mathematics nouns pronoun entertainment video
Topic #2:
military related topics variation locations operations aviation activities aircraft type
Topic #3:
culture music specifically number comparison symbol sports history german indicate
Topic #4:
time specific verb form activity period year event way location
Topic #5:
type kind organization concept tropical storms work nobility weather sports
Topic #6:
people places things history related culture events names politics language
Topic #7:
action movement concept place thing person idea verb specific state
Topic #8:
end letter different suffix mark letters united states pacific la
Topic #9:
names numbers measurements words process countries government architecture action law
Topic #10:
word end followed phrase contain noun start include transportation preposition


In [50]:
# Visualize clusters using PyLDAVis
pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda, dtm, vectorizer, mds='tsne')
pyLDAvis.save_html(vis, join(RESULTS_DIR, 'lda.html'))
vis

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
5     -10.303126 -25.569633       1        1  12.490453
9      57.620556 -55.138092       2        1  10.623635
3      36.481567  36.180618       3        1  10.565446
0      60.113197  -4.284040       4        1  10.346989
1      27.609348 -30.531887       5        1  10.147695
8       9.035104 -68.542763       6        1   9.526541
2     -15.805274  32.425098       7        1   9.314184
6      12.938659   4.869200       8        1   9.282137
4     -43.443966  -3.272498       9        1   9.039578
7     -36.099892 -55.070850      10        1   8.663344, topic_info=           Term        Freq       Total Category  logprob  loglift
2238       word  374.000000  374.000000  Default  30.0000  30.0000
705         end  472.000000  472.000000  Default  29.0000  29.0000
1501     people  344.000000  344.000000  Default  28.0000  28.0000
1538     places  247.000000  247.000000  Default  27.0000  27.0000
1303   military  211.000000  211.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
2244      world   10.735964   21.575482  Topic10  -5.1310   1.7481
2135      types   12.364603   38.043752  Topic10  -4.9898   1.3222
1460  ownership   10.423878   19.661284  Topic10  -5.1605   1.8115
881     germany    9.919509   17.952754  Topic10  -5.2101   1.8528
990        idea    9.075801   89.753287  Topic10  -5.2990   0.1546

[514 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
21        2  0.914100  abbreviations
27        2  0.887306             ac
46        3  0.175117         action
46        6  0.168863         action
46        8  0.650435         action
...     ...       ...            ...
2244      9  0.046349          world
2244     10  0.509838          world
2247      9  0.207268        writing
2247     10  0.759983        writing
2252      3  0.987825           year

[687 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 10, 4, 1, 2, 9, 3, 7, 5, 8])